In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_ds, test_ds = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTO6D17/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTO6D17/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTO6D17/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data = []
train_labels = []

for example, label in train_ds:
  train_data.append(example.numpy().decode('utf-8'))
  train_labels.append(label.numpy())


test_data = []
test_labels = []

for example, label in test_ds:
  test_data.append(example.numpy().decode('utf-8'))
  test_labels.append(label.numpy())

train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

In [ ]:
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=200, padding='post')
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=200, padding='post')

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.97:
      print('Stopping training.')
      self.model.stop_training = True

callback = MyCallback()

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(10000, 32, input_length=200, name='embedding'),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_sequences, train_labels, epochs=10, callbacks=[callback])

Epoch 1/10
782/782 [==============================] - 310s 389ms/step - loss: 0.4798 - accuracy: 0.7620
Epoch 2/10
782/782 [==============================] - 303s 387ms/step - loss: 0.3727 - accuracy: 0.8426
Epoch 3/10
782/782 [==============================] - 303s 387ms/step - loss: 0.5057 - accuracy: 0.7542
Epoch 4/10
782/782 [==============================] - 304s 389ms/step - loss: 0.3787 - accuracy: 0.8380
Epoch 5/10
782/782 [==============================] - 304s 389ms/step - loss: 0.3292 - accuracy: 0.8613
Epoch 6/10
782/782 [==============================] - 303s 388ms/step - loss: 0.3040 - accuracy: 0.8738
Epoch 7/10
782/782 [==============================] - 304s 389ms/step - loss: 0.2925 - accuracy: 0.8806
Epoch 8/10
782/782 [==============================] - 304s 389ms/step - loss: 0.2770 - accuracy: 0.8856
Epoch 9/10
782/782 [==============================] - 304s 389ms/step - loss: 0.2663 - accuracy: 0.8916
Epoch 10/10
782/782 [==============================] - 305s 390m

In [ ]:
correct = [0]
incorrect = [0]

def predict(review, label):

  review = review.reshape(200, 1)
  prediction = model.predict(review)

  if prediction.argmax() < 0.5:
    prediction = 0
  else:
    prediction = 1

  print('prediction: ', prediction)
  print('Label: ', label)

  if prediction - label == 0:
    print('Correct!')
    correct[0] +=1
  else:
    print('Incorrect!')
    incorrect[0] +=1

In [ ]:
for _ in range(1000):
  i = random.randint(0, len(test_data) - 1)
  predict(review=test_sequences[i], label=test_labels[i])

print(correct)
print(incorrect)

test_accuracy = round(correct[0] / (correct[0] + incorrect[0]), ndigits=2)
test_accuracy

prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  0
Label:  1
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  1
Correct!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  0
Incorrect!
prediction:  1
Label:  1
Correct!
prediction:  1
Label

0.54